In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
# Figure out how to import regularizers
from tensorflow.keras import regularizers
###
import tensorflow.keras.utils as ku 
from tensorflow import expand_dims
import numpy as np

In [1]:
# sonnets.txt
# !gdown --id 108jAePKK4R3BVYBbYJZ32JWUwxeMg20K

data = open('./sonnets.txt').read()

corpus = data.lower().split("\n")

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus) # Obtenemos los tokens para todas las palabras en el archivo
total_words = len(tokenizer.word_index) + 1 # A partir de aquí, ya tenemos un índice de palabra con su token asociado
total_words

3211

In [5]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	# Aquí convertimos cada texto a secuencias de texto, básicamente hacemos listas de palabras dentro de una lista más
	# grande pero reemplazando las palabras por tokens
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence) # Aquí obtenemos todas las longitudes posibles para cada línea, de tal
		# manera que podamos entrenar con todas las variantes, por ejemplo:
		"""
		Inicial: Hola a todos
		Tamaño 2: Ho
		Tamaño 3: Hol
		Tamaño 4: Hola
		...
		"""

In [6]:
input_sequences[0]

[34, 417]

In [7]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [8]:
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [22]:

"""
Construye una red neuronal que utilice al menos:
1 capa de Embedding
1 capa de LSTM Bidirectional
1 capa de Dropout
1 capa de LSTM
1 capa oculta con regularizadores de la siguiente forma:

model.add(Dense(total_words/2,
				kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
				bias_regularizer=regularizers.l2(1e-4),
				activity_regularizer=regularizers.l2(1e-5),
				activation='relu'))
    
1 capa de salida de tamaño total_words
"""
### START CODE HERE
model = Sequential()

model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dropout(0.10))
model.add(Lambda(lambda x: expand_dims(x, axis=-1)))
model.add(LSTM(100))
model.add(Dense(total_words/2,
				kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
				bias_regularizer=regularizers.l2(1e-4),
				activity_regularizer=regularizers.l2(1e-5),
				activation='relu'))
model.add(Dropout(0.10))
model.add(Dense(total_words, activation='softmax'))

# Pick an optimizer

opt = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

### END CODE HERE

print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 10, 100)           321100    
                                                                 
 bidirectional_11 (Bidirecti  (None, 300)              301200    
 onal)                                                           
                                                                 
 dropout_11 (Dropout)        (None, 300)               0         
                                                                 
 lambda (Lambda)             (None, 300, 1)            0         
                                                                 
 lstm_19 (LSTM)              (None, 100)               40800     
                                                                 
 dense_14 (Dense)            (None, 1605)              162105    
                                                      

In [ ]:
history = model.fit(predictors, label, epochs=100, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
"""
Construye un ciclo que te permita tokenizar la frase de entrada (seed_text) y obtener
la cantidad de palabras siguientes dada por next_words

Recuerda utilizar model.predict y las funciones text to sequences y pad_sequences
"""
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100

for _ in range(next_words):
	token_list = # YOUR CODE HERE
	token_list = # YOUR CODE HERE
	predicted = # YOUR CODE HERE
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word

print(seed_text)